In [ ]:
# @title 1. Install Dependencies
!pip install -q -U bitsandbytes
!pip install -q -U transformers accelerate
!pip install -q -U imagehash sentence-transformers

print("Dependencies installed.")

In [ ]:
# @title 2. Loading Model and Defining Logic Engine
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration, pipeline, BitsAndBytesConfig
from sentence_transformers import CrossEncoder
from PIL import Image, ImageChops, ImageEnhance
import spacy
import io
import imagehash
import base64
import warnings

warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

class NewsVerifier:
    def __init__(self):
        print("Loading Forensics & Vision Models...")
        self.fake_detector = pipeline("image-classification", model="umm-maybe/AI-image-detector", device=0 if device == "cuda" else -1)

        self.processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

        #  Quantizing it by 4-bit
        try:
            print("   ...Attempting 4-bit quantization load...")
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4"
            )
            self.model = Blip2ForConditionalGeneration.from_pretrained(
                "Salesforce/blip2-opt-2.7b",
                quantization_config=bnb_config,
                device_map="auto"
            )
            print("SUCCESS: 4-Bit Quantization Enabled!")
        except Exception as e:
            print(f"" 4-BIT FAILED with error: {e}")
            print("   ...Falling back to standard float16 loading.")
            self.model = Blip2ForConditionalGeneration.from_pretrained(
                "Salesforce/blip2-opt-2.7b",
                torch_dtype=torch.float16,
                device_map="auto"
            )

        print("Loading Logic/NLI Model...")
        self.nli_model = CrossEncoder('cross-encoder/nli-distilroberta-base', device=0)

        print("Loading NLP...")
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except OSError:
            from spacy.cli import download
            download("en_core_web_sm")
            self.nlp = spacy.load("en_core_web_sm")

        print("Systems Online.")
        self.known_fakes_db = {
            "d4d4d4d4d4d4d4d4": "Stock Photo: 'War Zone' (2015)",
            "a1a1a1a1a1a1a1a1": "Viral Hoax: 'Shark on Highway' (2018)",
        }

    def perform_ela(self, image):
        """Generates ELA Heatmap"""
        temp_buffer = io.BytesIO()
        image.convert("RGB").save(temp_buffer, format="JPEG", quality=90)
        temp_buffer.seek(0)

        ela_image = ImageChops.difference(image.convert("RGB"), Image.open(temp_buffer))
        extrema = ela_image.getextrema()
        max_diff = max([ex[1] for ex in extrema]) or 1
        scale = 255.0 / max_diff
        ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

        img_byte_arr = io.BytesIO()
        ela_image.save(img_byte_arr, format="PNG")
        encoded_image = base64.b64encode(img_byte_arr.getvalue()).decode('ascii')
        return f"data:image/png;base64,{encoded_image}"

    def detect_deepfake(self, image):
        try:
            results = self.fake_detector(image)
            for r in results:
                if r['label'] == 'artificial':
                    return round(r['score'] * 100, 2)
            return 0.0
        except: return 0.0

    def extract_claims(self, text):
        doc = self.nlp(text)
        entities = {"GPE": [], "DATE": [], "ORG": []}
        for ent in doc.ents:
            if ent.label_ in entities:
                entities[ent.label_].append(ent.text)
        return entities

    def ask_vqa(self, image, question):
        prompt = f"Question: {question} Answer:"
        inputs = self.processor(image, text=prompt, return_tensors="pt").to(device, torch.float16 if device == "cuda" else torch.float32)

        generated_ids = self.model.generate(**inputs, max_new_tokens=40, min_length=2, do_sample=False)
        output = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        return output.split("Answer:")[-1].strip() if "Answer:" in output else output

    def verify(self, image_bytes, text_caption):
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        flags = []

        #  HASH CHECK
        img_hash = str(imagehash.phash(image))
        print(f"DEBUG: Hash={img_hash}")
        if img_hash in self.known_fakes_db:
            flags.append({"type": "Recycled Media", "severity": "Critical", "details": f"Known fake: {self.known_fakes_db[img_hash]}"})

        # DEEPFAKE CHECK
        ai_prob = self.detect_deepfake(image)
        if ai_prob > 70:
            flags.append({"type": "⚠️ AI Generated", "severity": "Critical", "details": f"{ai_prob}% probability of being AI."})

        # Call 1 for General Scene
        visual_summary = self.ask_vqa(image, "Describe the scene, action, and location in this image.")
        print(f"DEBUG: Summary = {visual_summary}")

        # Call 2 for Language
        lang_check = self.ask_vqa(image, "What language is written on the signs?")
        if "english" not in lang_check.lower() and len(lang_check) > 3:
             visual_summary += f". Text language appears to be {lang_check}."
             flags.append({"type": "Language Context", "severity": "Info", "details": f"Detected text language: {lang_check}"})

        # NLI
        scores = self.nli_model.predict([(text_caption, visual_summary)])[0]
        contradiction_score = scores[0]
        print(f"DEBUG: Contradiction = {contradiction_score}")

        label_mapping = ['contradiction', 'entailment', 'neutral']
        predicted_label = label_mapping[scores.argmax()]

        if predicted_label == 'contradiction' and contradiction_score > 0.4:
            flags.append({
                "type": "Logical Contradiction",
                "severity": "Critical",
                "details": f"Caption claims '{text_caption}', but visual evidence contradicts this (AI saw: '{visual_summary}')."
            })

        # LOCATION CHECK
        claims = self.extract_claims(text_caption)
        if claims["GPE"]:
            for loc in claims["GPE"]:
                if loc.lower() not in visual_summary.lower():
                    confirm = self.ask_vqa(image, f"Is this {loc}? Answer yes or no.")
                    if "no" in confirm.lower():
                        flags.append({
                            "type": "Location Mismatch",
                            "severity": "High",
                            "details": f"Text claims '{loc}', but visual analysis suggests otherwise."
                        })

        return {
            "ai_generated_probability": ai_prob,
            "ai_generated_caption": visual_summary,
            "detected_entities": claims,
            "inconsistencies": flags,
            "raw_vqa_checks": visual_summary,
            "ela_heatmap": self.perform_ela(image)
        }

verifier_engine = NewsVerifier()

In [ ]:
# @title 3. Setup FastAPI & Dashboard
from fastapi import FastAPI, File, UploadFile, Form, Request
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
import uvicorn
import nest_asyncio
import os
import json

app = FastAPI()

# temp dir
os.makedirs("templates", exist_ok=True)
templates = Jinja2Templates(directory="templates")

# HTML
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>VeritasLens | Media Forensics</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css" rel="stylesheet">
    <style>
        :root { --primary-color: #2c3e50; --accent-color: #3498db; }
        body { background-color: #f8f9fa; font-family: 'Segoe UI', Roboto, Helvetica, Arial, sans-serif; }

        .navbar { background-color: var(--primary-color); box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
        .navbar-brand { font-weight: 700; letter-spacing: 0.5px; }

        .card { border: none; border-radius: 12px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); }
        .card-header { background-color: white; border-bottom: 1px solid #eee; font-weight: 600; border-radius: 12px 12px 0 0 !important; }

        
        .meter-container { background: #e9ecef; border-radius: 20px; height: 30px; position: relative; overflow: hidden; }
        .meter-fill { height: 100%; border-radius: 20px; transition: width 1.0s ease-in-out; display: flex; align-items: center; justify-content: flex-end; padding-right: 15px; color: white; font-weight: bold; }
        .bg-safe { background: linear-gradient(90deg, #56ab2f, #a8e063); }
        .bg-warn { background: linear-gradient(90deg, #fceabb, #f8b500); color: #5d4037; }
        .bg-fake { background: linear-gradient(90deg, #cb2d3e, #ef473a); }

        
        .flag-card { border-left: 5px solid; background: white; padding: 15px; border-radius: 8px; margin-bottom: 12px; box-shadow: 0 2px 4px rgba(0,0,0,0.02); }
        .flag-critical { border-color: #dc3545; background-color: #fff5f5; }
        .flag-high { border-color: #fd7e14; background-color: #fffbf2; }
        .flag-info { border-color: #0dcaf0; background-color: #f0faff; }
        .badge-entity { background-color: #e9ecef; color: #495057; margin: 0 5px 5px 0; padding: 8px 12px; font-weight: 500; }
    </style>
</head>
<body>

<nav class="navbar navbar-dark mb-4">
    <div class="container">
        <span class="navbar-brand"><i class="fas fa-shield-alt me-2"></i>VeritasLens <small class="opacity-75 fw-light">| Forensic Dashboard</small></span>
    </div>
</nav>

<div class="container pb-5">
    <div class="row g-4">
        <div class="col-lg-4">
            <div class="card h-100">
                <div class="card-header py-3">1. Upload Evidence</div>
                <div class="card-body">
                    <form id="analyzeForm">
                        <div class="mb-3">
                            <label class="form-label text-muted small fw-bold">IMAGE SOURCE</label>
                            <input class="form-control" type="file" id="imageInput" accept="image/*" required>
                            <div class="mt-3 text-center bg-light rounded p-2" style="min-height: 200px; display: flex; align-items: center; justify-content: center;">
                                <img id="preview" class="img-fluid rounded d-none" style="max-height: 250px;">
                                <span id="placeholder-text" class="text-muted small">No image selected</span>
                            </div>
                        </div>

                        <div class="mb-4">
                            <label class="form-label text-muted small fw-bold">CLAIM / CAPTION</label>
                            <textarea class="form-control" id="captionInput" rows="3" placeholder="e.g. 'Breaking: Riot in downtown Madrid'"></textarea>
                        </div>

                        <button type="submit" class="btn btn-primary w-100 py-2 fw-bold shadow-sm">
                            <i class="fas fa-search me-2"></i>RUN FORENSIC ANALYSIS
                        </button>
                    </form>
                </div>
            </div>
        </div>

        <div class="col-lg-8">
            <div class="card h-100">
                <div class="card-header py-3 d-flex justify-content-between align-items-center">
                    <span>2. Analysis Report</span>
                    <span id="statusBadge" class="badge bg-secondary">Idle</span>
                </div>

                <div class="card-body">
                    <div id="loading" class="text-center d-none py-5">
                        <div class="spinner-border text-primary mb-3" role="status" style="width: 3rem; height: 3rem;"></div>
                        <h5 class="text-dark">Analyzing Content...</h5>
                        <p class="text-muted small">Performing ELA Forensic Scan • Checking AI artifacts • Verifying Logic</p>
                    </div>

                    <div id="resultsArea" class="d-none">

                        <div class="mb-4">
                            <div class="d-flex justify-content-between mb-2">
                                <span class="fw-bold small text-muted">AI GENERATION PROBABILITY</span>
                                <span id="aiScoreText" class="fw-bold">0%</span>
                            </div>
                            <div class="meter-container">
                                <div id="aiScoreBar" class="meter-fill bg-safe" style="width: 0%"></div>
                            </div>
                        </div>

                        <div class="card mb-4 border-warning" style="background: #fffdf5;">
                            <div class="card-body">
                                <h6 class="fw-bold text-muted small mb-3"><i class="fas fa-fingerprint me-2"></i>DIGITAL TAMPERING (ELA HEATMAP)</h6>
                                <div class="row align-items-center">
                                    <div class="col-md-4 text-center">
                                        <img id="elaImage" class="img-fluid rounded border shadow-sm" style="max-height: 150px;">
                                    </div>
                                    <div class="col-md-8">
                                        <p class="small text-muted mb-0">
                                            <strong>How to read this:</strong> This heatmap highlights "error levels" in JPEG compression.
                                            Uniform darkness/noise is good. <br>
                                            <span class="text-danger fw-bold">Bright white patches</span> (that don't match edges) often indicate spliced/pasted objects (Photoshop edits).
                                        </p>
                                    </div>
                                </div>
                            </div>
                        </div>

                        <div class="row">
                            <div class="col-md-7">
                                <h6 class="fw-bold text-muted small mb-3">INCONSISTENCY FLAGS</h6>
                                <div id="flagsList"></div>
                            </div>
                            <div class="col-md-5">
                                <h6 class="fw-bold text-muted small mb-3">AI VISUAL SUMMARY</h6>
                                <div class="p-3 bg-light rounded border mb-4">
                                    <i class="fas fa-robot text-primary me-2"></i>
                                    <span id="aiCaption" class="fst-italic text-dark small">...</span>
                                </div>
                                <h6 class="fw-bold text-muted small mb-2">DETECTED ENTITIES</h6>
                                <div id="entitiesList"></div>
                            </div>
                        </div>
                    </div>

                    <div id="emptyState" class="text-center py-5">
                        <i class="fas fa-microscope fa-3x text-muted mb-3 opacity-25"></i>
                        <p class="text-muted">Upload media to begin verification.</p>
                    </div>
                </div>
            </div>
        </div>
    </div>
</div>

<script>
    document.getElementById('imageInput').onchange = evt => {
        const [file] = document.getElementById('imageInput').files
        if (file) {
            document.getElementById('preview').src = URL.createObjectURL(file)
            document.getElementById('preview').classList.remove('d-none')
            document.getElementById('placeholder-text').classList.add('d-none')
        }
    }

    document.getElementById('analyzeForm').onsubmit = async (e) => {
        e.preventDefault();
        document.getElementById('loading').classList.remove('d-none');
        document.getElementById('resultsArea').classList.add('d-none');
        document.getElementById('emptyState').classList.add('d-none');
        document.getElementById('statusBadge').innerText = 'Processing...';
        document.getElementById('statusBadge').className = 'badge bg-warning text-dark';

        const formData = new FormData();
        formData.append("file", document.getElementById('imageInput').files[0]);
        formData.append("caption", document.getElementById('captionInput').value);

        try {
            const response = await fetch("/analyze", { method: "POST", body: formData });
            const data = await response.json();

            // ai score
            const score = data.ai_generated_probability;
            const bar = document.getElementById('aiScoreBar');
            const scoreText = document.getElementById('aiScoreText');
            bar.style.width = score + "%";
            scoreText.innerText = score + "%";
            if(score < 20) bar.className = "meter-fill bg-safe";
            else if (score < 70) bar.className = "meter-fill bg-warn";
            else bar.className = "meter-fill bg-fake";

            // heatmap
            document.getElementById('elaImage').src = data.ela_heatmap;

            // flags
            const flagsContainer = document.getElementById('flagsList');
            flagsContainer.innerHTML = '';
            if (data.inconsistencies.length === 0) {
                flagsContainer.innerHTML = `<div class="alert alert-success d-flex align-items-center"><i class="fas fa-check-circle fa-2x me-3"></i><div><strong>Verified</strong><br><small>Visuals align with the text claim.</small></div></div>`;
            } else {
                data.inconsistencies.forEach(flag => {
                    let cardClass = 'flag-info';
                    let icon = 'fa-info-circle';

                    
                    if (flag.severity === 'Critical') { cardClass = 'flag-critical'; icon = 'fa-exclamation-triangle'; }
                    else if (flag.severity === 'High') { cardClass = 'flag-high'; icon = 'fa-exclamation-circle'; }

                    
                    if (flag.type.includes('Recycled')) icon = 'fa-recycle';
                    if (flag.type.includes('Location')) icon = 'fa-map-marker-alt';
                    if (flag.type.includes('AI')) icon = 'fa-robot';
                    if (flag.type.includes('Language')) icon = 'fa-language';
                    if (flag.type.includes('Logical')) icon = 'fa-brain';

                    const div = document.createElement('div');
                    div.className = `flag-card ${cardClass}`;
                    div.innerHTML = `<div class="d-flex"><div class="me-3"><i class="fas ${icon} fa-lg"></i></div><div><strong class="text-uppercase small d-block mb-1">${flag.type}</strong><span class="small">${flag.details}</span></div></div>`;
                    flagsContainer.appendChild(div);
                });
            }

        
            document.getElementById('aiCaption').innerText = '"' + data.ai_generated_caption + '"';
            const entContainer = document.getElementById('entitiesList');
            entContainer.innerHTML = '';
            const entDict = data.detected_entities;
            let hasEnt = false;
            if(entDict.GPE) entDict.GPE.forEach(e => { entContainer.innerHTML += `<span class="badge badge-entity"><i class="fas fa-map-marker-alt me-1"></i>${e}</span>`; hasEnt = true; });
            if(entDict.DATE) entDict.DATE.forEach(e => { entContainer.innerHTML += `<span class="badge badge-entity"><i class="far fa-calendar-alt me-1"></i>${e}</span>`; hasEnt = true; });
            if(!hasEnt) entContainer.innerHTML = '<span class="text-muted small">No specific entities found.</span>';

            document.getElementById('loading').classList.add('d-none');
            document.getElementById('resultsArea').classList.remove('d-none');
            document.getElementById('statusBadge').innerText = 'Complete';
            document.getElementById('statusBadge').className = 'badge bg-success';

        } catch (err) {
            console.error(err);
            alert("Analysis failed. Check Colab logs for error.");
            document.getElementById('loading').classList.add('d-none');
            document.getElementById('statusBadge').innerText = 'Error';
            document.getElementById('statusBadge').className = 'badge bg-danger';
        }
    };
</script>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(html_code)

# API endpoints
@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/analyze")
async def analyze(file: UploadFile = File(...), caption: str = Form(...)):
    contents = await file.read()

    result = verifier_engine.verify(contents, caption)
    return result

print("Server and Dashboard code ready.")

In [ ]:
!pip install pyngrok

In [ ]:
# @title 4. Start Server
import threading
from pyngrok import ngrok
import uvicorn
import nest_asyncio

ngrok.kill()

NGROK_TOKEN = "3Get and enter your own"
ngrok.set_auth_token(NGROK_TOKEN)

try:
  public_url = ngrok.connect(8000).public_url
  print(f"\n Public URL: {public_url}\nClick the link above to open the dashboard.")
except Exception as e:
  print(f"Error starting tunnel: {e}")

def run_server():
  nest_asyncio.apply()
  uvicorn.run(app, port=8000, host="127.0.0.1", log_level="info")

thread = threading.Thread(target=run_server)
thread.start()